In [18]:
# MVP - Recherche <-> Erkenntnisse
## Nutzer im Mittelpunkt
## Vorzeigbar + Simple User Interface
## Algos stabil und performant
## Output klar ersichtlich


# Benotung
## Praktisch
### funtionierender Prototyp
### lauffähiger/ gut kommentierter/ dokumentierter Python Code
### Code Qualität

## Vorstellung
### Literaturrecherche
### Technische Architektur (+ Pakete, APIs)
### Performance der Modelle
### Verbesserungsvorschläge
### 45 Minuten


# Image Segmentation
## Challenge: Explore each type of image segmentation; Choose two types and evaluate each segmentation techniques with evaluation metrics
## Dataset: creative common images OR popular image datasets

## https://www.wikiwand.com/en/Image_segmentation

## semantic, instance, panoptic
## https://blog.roboflow.com/difference-semantic-segmentation-instance-segmentation/
## https://paperswithcode.com/task/semantic-segmentation
## https://paperswithcode.com/task/instance-segmentation
## Dataset: ADE20K
## https://groups.csail.mit.edu/vision/datasets/ADE20K/

In [19]:
## semantic segmentation
### no extra training data
### simple implementation first
### https://towardsdatascience.com/train-neural-net-for-semantic-segmentation-with-pytorch-in-50-lines-of-code-830c71a6544f
### Dataset (LabPics): https://zenodo.org/record/3697452

import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

Learning_Rate = 1e-5
width = height = 800 # image width and height
batchSize = 3

TrainFolder = "LabPicsV1/Simple/Train//"
ListImages = os.listdir(os.path.join(TrainFolder, "Image"))

In [20]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)), tf.ToTensor()])

In [21]:
def ReadRandomImage():
    idx=np.random.randint(0,len(ListImages)) # Pick random image
    Img=cv2.imread(os.path.join(TrainFolder, "Image",ListImages[idx]))
    Filled =  cv2.imread(os.path.join(TrainFolder,   "Semantic/16_Filled", ListImages[idx].replace("jpg","png")),0)

    Vessel =  cv2.imread(os.path.join(TrainFolder, "Semantic/1_Vessel", ListImages[idx].replace("jpg","png")),0)
    AnnMap = np.zeros(Img.shape[0:2],np.float32) # Segmentation map
    if Vessel is not None:  AnnMap[ Vessel == 1 ] = 1
    if Filled is not None:  AnnMap[ Filled  == 1 ] = 2
    Img=transformImg(Img)
    AnnMap=transformAnn(AnnMap)
    return Img,AnnMap

In [22]:
def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])

    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()

    return images, ann

In [54]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
Net.classifier[4] = torch.nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 3 classes
Net=Net.to(device)
optimizer=torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate,weight_decay=0) # Create adam optimizer

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [56]:
Pred = None
for itr in range(1): # Training loop
    images,ann=LoadBatch()

    images=torch.autograd.Variable(images,requires_grad=False).to(device)

    ann = torch.autograd.Variable(ann,requires_grad=False).to(device)

    Pred=Net(images)["out"] # make prediction

In [50]:
type(Pred.shape)
print(ann.long().shape)

torch.Size([3, 1000])
torch.Size([3, 800, 800])


In [57]:
criterion = torch.nn.CrossEntropyLoss() # Set loss function
Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
Loss.backward() # Backpropogate loss
optimizer.step() # Apply gradient descent change to weight

In [60]:
itr = 1
print("Saving Model" +str(itr) + ".torch")
torch.save(Net.state_dict(), str(itr) + ".torch")

Saving Model1.torch


In [62]:
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf
import matplotlib.pyplot as plt
modelPath = "1.torch"  # Path to trained model
imagePath = "test.jpg"  # Test image
height=width=900
transformImg = tf.Compose([tf.ToPILImage(), tf.Resize((height, width)), tf.ToTensor(),tf.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
Net.classifier[4] = torch.nn.Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
Net = Net.to(device)  # Set net to GPU or CPU
Net.load_state_dict(torch.load(modelPath)) # Load trained model
Net.eval() # Set to evaluation mode

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se